In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [9]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [10]:
# Check the unique value counts to see if binning is required
charity_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,NAME_146TH ALUMNI ASSOCIATION,NAME_14TH ARMORED DIVISION,NAME_150TH INTELLIGENCE SQUADRON ENLISTED ADVISORY COUNCIL,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Remove charity status target from features data
y = np.array(charity_df.SPECIAL_CONSIDERATIONS_Y)
X = charity_df.drop(columns=["SPECIAL_CONSIDERATIONS_Y","SPECIAL_CONSIDERATIONS_N"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.999


In [10]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 11s 416us/sample - loss: 0.0997 - accuracy: 0.9923
Epoch 2/50
25724/25724 [==============================] - 8s 313us/sample - loss: 0.0135 - accuracy: 0.9992
Epoch 3/50
25724/25724 [==============================] - 7s 260us/sample - loss: 0.0086 - accuracy: 0.9992
Epoch 4/50
25724/25724 [==============================] - 6s 247us/sample - loss: 0.0066 - accuracy: 0.9992
Epoch 5/50
25724/25724 [==============================] - 6s 252us/sample - loss: 0.0058 - accuracy: 0.9992
Epoch 6/50
25724/25724 [==============================] - 7s 269us/sample - loss: 0.0047 - accuracy: 0.9992
Epoch 7/50
25724/25724 [==============================] - 6s 247us/sample - loss: 0.0036 - accuracy: 0.9992
Epoch 8/50
25724/25724 [==============================] - 7s 255us/sample - loss: 0.0022 - accuracy: 0.9993
Epoch 9/50
25724/25724 [==============================] - 7s 255us/sample - loss: 0.0013 - accuracy: 0.9995
Epoc

In [11]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
  928/25724 [>.............................] - ETA: 37s - loss: 0.0015 - accuracy: 0.9989
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1920/25724 [=>............................] - ETA: 23s - loss: 7.2855e-04 - accuracy: 0.9995
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2912/25724 [==>...........................] - ETA: 17s - loss: 5.2229e-04 - accuracy: 0.9997
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4032/25724 [===>..........................] - ETA: 14s - loss: 3.7721e-04 - accuracy: 0.9998
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4960/25724 [====>.........................] - ETA: 12s - loss: 3.0663e-04 - accuracy: 0.9998
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6080/25724 [======>.......................] - ETA: 10s - loss: 2.5015e-04 - accuracy: 0.9998
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 7104/25724 [=======>......................] - ETA: 9s - loss: 

 3680/25724 [===>..........................] - ETA: 6s - loss: 5.0560e-10 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4736/25724 [====>.........................] - ETA: 6s - loss: 9.6820e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5792/25724 [=====>........................] - ETA: 5s - loss: 7.9168e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 6880/25724 [=======>......................] - ETA: 5s - loss: 6.6649e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7712/25724 [=======>......................] - ETA: 5s - loss: 5.9482e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8832/25724 [=========>....................] - ETA: 4s - loss: 5.1939e-07 - accuracy: 1.0000
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9888/25724 [==========>...................] - ETA: 4s - loss: 6.1439e-04 - accuracy: 0.9999
Epoch 0

 9344/25724 [=========>....................] - ETA: 4s - loss: 3.8075e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
10432/25724 [===========>..................] - ETA: 4s - loss: 3.4104e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
11392/25724 [============>.................] - ETA: 4s - loss: 3.1237e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
12384/25724 [=============>................] - ETA: 3s - loss: 2.9660e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
13408/25724 [==============>...............] - ETA: 3s - loss: 2.7395e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
14432/25724 [===============>..............] - ETA: 3s - loss: 2.5451e-04 - accuracy: 0.9999
Epoch 00007: saving model to checkpoints/weights.07.hdf5
15424/25724 [================>.............] - ETA: 2s - loss: 2.3815e-04 - accuracy: 0.9999
Epoch 0

15072/25724 [================>.............] - ETA: 2s - loss: 2.4563e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
15904/25724 [=================>............] - ETA: 2s - loss: 2.3513e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
17024/25724 [==================>...........] - ETA: 2s - loss: 2.1966e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
18080/25724 [====================>.........] - ETA: 1s - loss: 2.0683e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
19136/25724 [=====================>........] - ETA: 1s - loss: 1.9566e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
20192/25724 [======================>.......] - ETA: 1s - loss: 1.8543e-04 - accuracy: 0.9999
Epoch 00011: saving model to checkpoints/weights.11.hdf5
21216/25724 [=======================>......] - ETA: 1s - loss: 1.7648e-04 - accuracy: 0.9999
Epoch 0

20576/25724 [======================>.......] - ETA: 1s - loss: 7.2294e-07 - accuracy: 1.0000
Epoch 00015: saving model to checkpoints/weights.15.hdf5
21568/25724 [========================>.....] - ETA: 1s - loss: 6.8969e-07 - accuracy: 1.0000
Epoch 00015: saving model to checkpoints/weights.15.hdf5
22624/25724 [=========================>....] - ETA: 0s - loss: 2.3196e-04 - accuracy: 1.0000
Epoch 00015: saving model to checkpoints/weights.15.hdf5
23648/25724 [==========================>...] - ETA: 0s - loss: 2.2255e-04 - accuracy: 1.0000
Epoch 00015: saving model to checkpoints/weights.15.hdf5
24704/25724 [===========================>..] - ETA: 0s - loss: 2.5306e-04 - accuracy: 0.9999
Epoch 00015: saving model to checkpoints/weights.15.hdf5
25724/25724 [==============================] - 7s 263us/sample - loss: 2.4303e-04 - accuracy: 0.9999
Epoch 16/100
   32/25724 [..............................] - ETA: 7s - loss: 1.4711e-10 - accuracy: 1.0000
Epoch 00016: saving model to checkpoints/we

...

  928/25724 [>.............................] - ETA: 5s - loss: 8.9193e-12 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 1984/25724 [=>............................] - ETA: 5s - loss: 6.0131e-12 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 3040/25724 [==>...........................] - ETA: 5s - loss: 1.7464e-11 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 3872/25724 [===>..........................] - ETA: 5s - loss: 1.5228e-11 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 4992/25724 [====>.........................] - ETA: 5s - loss: 1.2637e-11 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 6048/25724 [======>.......................] - ETA: 4s - loss: 1.4232e-11 - accuracy: 1.0000
Epoch 00081: saving model to checkpoints/weights.81.hdf5
 7104/25724 [=======>......................] - ETA: 4s - loss: 1.3756e-11 - accuracy: 1.0000
Epoch 0

 6528/25724 [======>.......................] - ETA: 4s - loss: 2.8520e-04 - accuracy: 0.9998
Epoch 00085: saving model to checkpoints/weights.85.hdf5
 7584/25724 [=======>......................] - ETA: 4s - loss: 2.4549e-04 - accuracy: 0.9999
Epoch 00085: saving model to checkpoints/weights.85.hdf5
 8640/25724 [=========>....................] - ETA: 4s - loss: 2.1558e-04 - accuracy: 0.9999
Epoch 00085: saving model to checkpoints/weights.85.hdf5
 9696/25724 [==========>...................] - ETA: 4s - loss: 1.9210e-04 - accuracy: 0.9999
Epoch 00085: saving model to checkpoints/weights.85.hdf5
10528/25724 [===========>..................] - ETA: 3s - loss: 1.7692e-04 - accuracy: 0.9999
Epoch 00085: saving model to checkpoints/weights.85.hdf5
11648/25724 [============>.................] - ETA: 3s - loss: 1.5991e-04 - accuracy: 0.9999
Epoch 00085: saving model to checkpoints/weights.85.hdf5
12704/25724 [=============>................] - ETA: 3s - loss: 1.4661e-04 - accuracy: 0.9999
Epoch 0

12160/25724 [=============>................] - ETA: 3s - loss: 4.8959e-08 - accuracy: 1.0000
Epoch 00089: saving model to checkpoints/weights.89.hdf5
13248/25724 [==============>...............] - ETA: 3s - loss: 4.4945e-08 - accuracy: 1.0000
Epoch 00089: saving model to checkpoints/weights.89.hdf5
14304/25724 [===============>..............] - ETA: 2s - loss: 4.1628e-08 - accuracy: 1.0000
Epoch 00089: saving model to checkpoints/weights.89.hdf5
15136/25724 [================>.............] - ETA: 2s - loss: 5.9720e-05 - accuracy: 0.9999
Epoch 00089: saving model to checkpoints/weights.89.hdf5
16288/25724 [=================>............] - ETA: 2s - loss: 5.5496e-05 - accuracy: 0.9999
Epoch 00089: saving model to checkpoints/weights.89.hdf5
17344/25724 [===================>..........] - ETA: 2s - loss: 5.2117e-05 - accuracy: 0.9999
Epoch 00089: saving model to checkpoints/weights.89.hdf5
18400/25724 [====================>.........] - ETA: 1s - loss: 4.9126e-05 - accuracy: 0.9999
Epoch 0

17664/25724 [===================>..........] - ETA: 1s - loss: 7.1114e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
18688/25724 [====================>.........] - ETA: 1s - loss: 6.7218e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
19904/25724 [======================>.......] - ETA: 1s - loss: 9.8219e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
20832/25724 [=======================>......] - ETA: 1s - loss: 9.3843e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
21760/25724 [========================>.....] - ETA: 0s - loss: 8.9874e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
22784/25724 [=========================>....] - ETA: 0s - loss: 8.5834e-05 - accuracy: 0.9999
Epoch 00093: saving model to checkpoints/weights.93.hdf5
24032/25724 [===========================>..] - ETA: 0s - loss: 8.1406e-05 - accuracy: 0.9999
Epoch 0

23392/25724 [==========================>...] - ETA: 0s - loss: 6.5764e-05 - accuracy: 1.0000
Epoch 00097: saving model to checkpoints/weights.97.hdf5
24544/25724 [===========================>..] - ETA: 0s - loss: 6.2677e-05 - accuracy: 1.0000
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25376/25724 [============================>.] - ETA: 0s - loss: 6.0622e-05 - accuracy: 1.0000
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 6s 239us/sample - loss: 5.9802e-05 - accuracy: 1.0000
Epoch 98/100
  736/25724 [..............................] - ETA: 5s - loss: 5.3642e-14 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
 1856/25724 [=>............................] - ETA: 5s - loss: 5.1835e-12 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
 2688/25724 [==>...........................] - ETA: 5s - loss: 3.5820e-12 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/we

In [15]:
# Preprocess numerical data for neural network

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [18]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.999
